#  Create a custom covariance matrix

In [1]:
import sandy

In [2]:
import pandas as pd
import numpy as np
import tempfile
import os

pd.options.display.float_format = '{:.5e}'.format

This notebook explains how to create a custom covariance matrix for a energy-dependent cross section.
In this example only one material (MT) and one reaction (MT) are used, but the exercise can be extended to other combinations. 

First, we define an energy grid for which intervals are uniformly distributed on a logarithmic scale.

In [3]:
size = 49

egrid = sandy.uniform_loggrid(1e-5, 1e7, size)
egrid

array([1.00000000e-05, 1.77827941e-05, 3.16227766e-05, 5.62341325e-05,
       1.00000000e-04, 1.77827941e-04, 3.16227766e-04, 5.62341325e-04,
       1.00000000e-03, 1.77827941e-03, 3.16227766e-03, 5.62341325e-03,
       1.00000000e-02, 1.77827941e-02, 3.16227766e-02, 5.62341325e-02,
       1.00000000e-01, 1.77827941e-01, 3.16227766e-01, 5.62341325e-01,
       1.00000000e+00, 1.77827941e+00, 3.16227766e+00, 5.62341325e+00,
       1.00000000e+01, 1.77827941e+01, 3.16227766e+01, 5.62341325e+01,
       1.00000000e+02, 1.77827941e+02, 3.16227766e+02, 5.62341325e+02,
       1.00000000e+03, 1.77827941e+03, 3.16227766e+03, 5.62341325e+03,
       1.00000000e+04, 1.77827941e+04, 3.16227766e+04, 5.62341325e+04,
       1.00000000e+05, 1.77827941e+05, 3.16227766e+05, 5.62341325e+05,
       1.00000000e+06, 1.77827941e+06, 3.16227766e+06, 5.62341325e+06,
       1.00000000e+07])

Then, we create a `pandas.MultiIndex` instance to use as the `index` and `columns` of the covariance matrix object. 

We resort to method `from_product` to make the `pd.MultiIndex` from the cartesian product of the MAT number, the MT number and the energy grid.

In [4]:
mat = 9437
mt = 102
index = pd.MultiIndex.from_product(([mat], [mt], egrid), names=["MAT", "MT", "E"])

To simplify the problem, we create a covariance matrix without correlations and with a constante variance.  

In [5]:
std = 3 / 100
data = np.eye(index.size) * std**2 
cov = sandy.CategoryCov(data, index=index, columns=index)
cov.data.head()

MAT                         9437                                      \
MT                           102                                       
E                    1.00000e-05 1.77828e-05 3.16228e-05 5.62341e-05   
MAT  MT  E                                                             
9437 102 1.00000e-05 9.00000e-04 0.00000e+00 0.00000e+00 0.00000e+00   
         1.77828e-05 0.00000e+00 9.00000e-04 0.00000e+00 0.00000e+00   
         3.16228e-05 0.00000e+00 0.00000e+00 9.00000e-04 0.00000e+00   
         5.62341e-05 0.00000e+00 0.00000e+00 0.00000e+00 9.00000e-04   
         1.00000e-04 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00   

MAT                                                                   \
MT                                                                     
E                    1.00000e-04 1.77828e-04 3.16228e-04 5.62341e-04   
MAT  MT  E                                                             
9437 102 1.00000e-05 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00   
         1.77828e-05 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00   
         3.16228e-05 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00   
         5.62341e-05 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00   
         1.00000e-04 9.00000e-04 0.00000e+00 0.00000e+00 0.00000e+00   

MAT                                           ...                          \
MT                                            ...                           
E                    1.00000e-03 1.77828e-03  ... 5.62341e+04 1.00000e+05   
MAT  MT  E                                    ...                           
9437 102 1.00000e-05 0.00000e+00 0.00000e+00  ... 0.00000e+00 0.00000e+00   
         1.77828e-05 0.00000e+00 0.00000e+00  ... 0.00000e+00 0.00000e+00   
         3.16228e-05 0.00000e+00 0.00000e+00  ... 0.00000e+00 0.00000e+00   
         5.62341e-05 0.00000e+00 0.00000e+00  ... 0.00000e+00 0.00000e+00   
         1.00000e-04 0.00000e+00 0.00000e+00  ... 0.00000e+00 0.00000e+00   

MAT                                                                   \
MT                                                                     
E                    1.77828e+05 3.16228e+05 5.62341e+05 1.00000e+06   
MAT  MT  E                                                             
9437 102 1.00000e-05 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00   
         1.77828e-05 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00   
         3.16228e-05 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00   
         5.62341e-05 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00   
         1.00000e-04 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00   

MAT                                                                   
MT                                                                    
E                    1.77828e+06 3.16228e+06 5.62341e+06 1.00000e+07  
MAT  MT  E                                                            
9437 102 1.00000e-05 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00  
         1.77828e-05 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00  
         3.16228e-05 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00  
         5.62341e-05 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00  
         1.00000e-04 0.00000e+00 0.00000e+00 0.00000e+00 0.00000e+00  

[5 rows x 49 columns]

> By using `CategoryCov` instead of `EnergyCov` we accept any sort of index.

The covariance matrix can be written to a `.csv` file for further use.

In [6]:
with tempfile.TemporaryDirectory() as td:
    csvfile = os.path.join(td, "custom_cov.csv")
    cov.data.to_csv(csvfile)
    
    df = sandy.CategoryCov.from_csv(csvfile, index_col=[0, 1, 2], header=[0, 1, 2]).data.head()
df

TypeError: 'classmethod' object is not callable

To reload the matrix from csv, make sure to specify `index_col` and `header` to account for the multiindex.

The same can be done also when there is more than one reaction or nuclide.

In [ ]:
mat = 9437
mts = [18, 102]
index = pd.MultiIndex.from_product(([mat], mts, egrid), names=["MAT", "MT", "E"])

In [ ]:
std = 3 / 100
data = np.eye(index.size) * std**2 
cov = sandy.CategoryCov(data, index=index, columns=index)
cov.data.head()